In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import torch
print("pytorch ver . ",torch.__version__)
print("Is CUDA available?",torch.cuda.is_available())
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader
import torch.utils.data as Data
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import xarray as xr
import os
os.environ['R_HOME'] = '/home/luhung3080/miniconda3/envs/chou/lib/R'
from rpy2.robjects import r, numpy2ri
numpy2ri.activate()
from rpy2.robjects.packages import importr
sinkr = importr('sinkr')
import Nio
print ("pynio ver . ",Nio.__version__)

/home/luhung3080/miniconda3/envs/chou/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


pytorch ver .  1.11.0+cu113
Is CUDA available? True
pynio ver .  1.5.5


In [2]:
data=pd.read_csv('v100_hr_20200101-0930_cut_utf8.csv')

/home/luhung3080/miniconda3/envs/chou/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3524: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## DINEOF

In [3]:
###
#cal_PMf  
###
u=np.zeros([244,6816])
for i in range (0,244):
    a=np.array(data['cal_PMf'][6816*i:6816*i+6816])
    for j in range(0,6816):
        if a[j]=='\\N' :
            a[j]=np.nan
    u[i]=a.T

###
#obs_PMf
###
v=np.zeros([244,6816])
for i in range (0,244):
    a=np.array(data['obs_PMf'][6816*i:6816*i+6816])
    for j in range(0,6816):
        if a[j]=='\\N' :
            a[j]=np.nan
    v[i]=a.T

print(np.shape(u))
print(np.shape(v))

(244, 6816)
(244, 6816)


In [4]:
XRestruct=sinkr.dineof(u)
YRestruct=sinkr.dineof(v)

[1] "1 EOF ; RMS = 10.22778933"
[1] "1 EOF ; RMS = 10.21055017"
[1] "1 EOF ; RMS = 10.21013413"
[1] "1 EOF ; RMS = 10.21015504"
[1] "2 EOF ; RMS = 9.34632169"
[1] "2 EOF ; RMS = 9.34479994"
[1] "2 EOF ; RMS = 9.34481473"
[1] "3 EOF ; RMS = 8.99367143"
[1] "3 EOF ; RMS = 8.99303692"
[1] "3 EOF ; RMS = 8.99271312"
[1] "3 EOF ; RMS = 8.99264261"
[1] "3 EOF ; RMS = 8.99264141"
[1] "4 EOF ; RMS = 8.80085908"
[1] "4 EOF ; RMS = 8.81074702"
[1] "5 EOF ; RMS = 8.68874813"
[1] "5 EOF ; RMS = 8.70179045"
[1] "6 EOF ; RMS = 8.55522702"
[1] "6 EOF ; RMS = 8.56073852"
[1] "7 EOF ; RMS = 8.49728582"
[1] "7 EOF ; RMS = 8.51513219"
[1] "8 EOF ; RMS = 8.43088817"
[1] "8 EOF ; RMS = 8.45416035"
[1] "9 EOF ; RMS = 8.42193151"
[1] "9 EOF ; RMS = 8.44935172"
[1] "10 EOF ; RMS = 8.46565639"
[1] "1 EOF ; RMS = 8.25477747"
[1] "1 EOF ; RMS = 8.20677729"
[1] "1 EOF ; RMS = 8.20511392"
[1] "1 EOF ; RMS = 8.20494421"
[1] "1 EOF ; RMS = 8.20491815"
[1] "1 EOF ; RMS = 8.20491348"
[1] "2 EOF ; RMS = 7.5594069"
[1] 

In [5]:
XRestruct_Fun=np.array(XRestruct[0])
YRestruct_Fun=np.array(YRestruct[0])
XCov_fun=np.cov(XRestruct_Fun.T)
XMean_fun=np.zeros(6816)
YMean_fun=np.zeros(6816)
for i in range(0,6816):
    for j in range(0,244):
        XMean_fun[i]=XMean_fun[i]+(XRestruct_Fun[j][i]/244)
        YMean_fun[i]=YMean_fun[i]+(YRestruct_Fun[j][i]/244)
print(type(XRestruct_Fun))
print(type(YRestruct_Fun))
print(np.shape(XRestruct_Fun))
print(np.shape(YRestruct_Fun))
print(np.shape(XCov_fun))
print(np.shape(XMean_fun))
print(np.shape(YMean_fun))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(244, 6816)
(244, 6816)
(6816, 6816)
(6816,)
(6816,)


In [6]:
Xhat=XRestruct_Fun
Yhat=YRestruct_Fun
Xbar=XMean_fun
Ybar=YMean_fun
print('Xhat')
#print(Xhat)
print(np.shape(Xhat))
print('Yhat')
#print(Yhat)
print(np.shape(Yhat))
print('Xbar')
#print(Xbar)
print(np.shape(Xbar))
print('Ybar')
#print(Ybar)
print(np.shape(Ybar))

Xhat
(244, 6816)
Yhat
(244, 6816)
Xbar
(6816,)
Ybar
(6816,)


In [7]:
G_xx=XCov_fun
G_xxt=np.zeros(6816)
for i in range(0,6816):
     G_xxt[i]=G_xx[i][i]
G_xy=0
for i in range (0,244):
    G_xyi=(Xhat[i]-Xbar)*(Yhat[i]-Ybar)
    G_xy=G_xy+G_xyi
G_xy=G_xy/243

print(G_xxt)
print(np.shape(G_xxt))
print(G_xy)
print(np.shape(G_xy))

[ 68.15802914  63.24933945 116.31025274 ...  98.99547747 118.30559943
 139.82751368]
(6816,)
[40.6237598  41.50159431 47.97859119 ... 42.2360388  51.81930767
 68.43299211]
(6816,)


In [8]:
B_1=G_xy/G_xxt
B_0=YMean_fun-(B_1*XMean_fun)

print('B_1')
#print(B_1)
print(np.shape(B_1))
print('B_0')
#print(B_0)
print(np.shape(B_0))

B_1
(6816,)
B_0
(6816,)


In [9]:
before_fx=np.zeros([274,6816])
for i in range (0,274):
    a=np.array(data['cal_PMf'][6816*i:6816*i+6816])
    for j in range(0,6816):
        if a[j]=='\\N' :
            a[j]=np.nan
    before_fx[i]=a
print(np.shape(before_fx))

(274, 6816)


In [10]:
after_fx_r=sinkr.dineof(before_fx)
after_fx=np.array(after_fx_r[0])
print(np.shape(after_fx))

[1] "1 EOF ; RMS = 10.07923554"
[1] "1 EOF ; RMS = 10.06407105"
[1] "1 EOF ; RMS = 10.0631579"
[1] "1 EOF ; RMS = 10.06297708"
[1] "1 EOF ; RMS = 10.06292945"
[1] "1 EOF ; RMS = 10.06291588"
[1] "1 EOF ; RMS = 10.06291195"
[1] "2 EOF ; RMS = 9.27140516"
[1] "2 EOF ; RMS = 9.27161068"
[1] "3 EOF ; RMS = 8.92975587"
[1] "3 EOF ; RMS = 8.92911273"
[1] "3 EOF ; RMS = 8.92925706"
[1] "4 EOF ; RMS = 8.7072588"
[1] "4 EOF ; RMS = 8.71244806"
[1] "5 EOF ; RMS = 8.55984651"
[1] "5 EOF ; RMS = 8.56905402"
[1] "6 EOF ; RMS = 8.45198387"
[1] "6 EOF ; RMS = 8.46277685"
[1] "7 EOF ; RMS = 8.36779568"
[1] "7 EOF ; RMS = 8.37534963"
[1] "8 EOF ; RMS = 8.28076509"
[1] "8 EOF ; RMS = 8.28329056"
[1] "9 EOF ; RMS = 8.21130365"
[1] "9 EOF ; RMS = 8.22016068"
[1] "10 EOF ; RMS = 8.15867437"
[1] "10 EOF ; RMS = 8.17101525"
[1] "11 EOF ; RMS = 8.10596557"
[1] "11 EOF ; RMS = 8.12081115"
[1] "12 EOF ; RMS = 8.10500618"
[1] "12 EOF ; RMS = 8.12934121"
[1] "13 EOF ; RMS = 8.10160653"
[1] "13 EOF ; RMS = 8.12190

In [11]:
fit_fx=B_0+(B_1*after_fx)
print(np.shape(fit_fx))

(274, 6816)


In [12]:
"""
old data
"""
old_x=np.zeros([274,6816])
old_y=np.zeros([274,6816])
#data_cal
for i in range (0,274):
    a=np.array(data['cal_PMf'][6816*i:6816*i+6816])
    for j in range(0,6816):
        if a[j]=='\\N' :
            a[j]=np.nan
    old_x[i]=a
#data_obs
for i in range (0,274):
    a=np.array(data['obs_PMf'][6816*i:6816*i+6816])
    for j in range(0,6816):
        if a[j]=='\\N' :
            a[j]=np.nan
    old_y[i]=a
print(np.shape(old_x))
print(np.shape(old_y))
print(np.shape(fit_fx))

(274, 6816)
(274, 6816)
(274, 6816)


In [13]:
old_x1=np.zeros([244,6816])
old_y1=np.zeros([244,6816])
fit_fx1=np.zeros([244,6816])
for i in range (0,244):
    for j in range (0,6816):
        old_x1[i][j]=old_x[i][j]
        old_y1[i][j]=old_y[i][j]
        fit_fx1[i][j]=fit_fx[i][j]
before1=abs(old_x1-old_y1)
after1=abs(fit_fx1-old_y1)
before_error1=np.nanmean(before1)
after_error1=np.nanmean(after1)
print("January to August (training data set) before_error (MADE) : %f" %before_error1)
print("January to August (training data set) after_error (MADE) : %f" %after_error1)

January to August (training data set) before_error (MADE) : 8.247546
January to August (training data set) after_error (MADE) : 6.792790


In [15]:
old_x2=np.zeros([30,6816])
old_y2=np.zeros([30,6816])
fit_fx2=np.zeros([30,6816])
for i in range (0,30):
    for j in range (0,6816):
        old_x2[i][j]=old_x[i+244][j]
        old_y2[i][j]=old_y[i+244][j]
        fit_fx2[i][j]=fit_fx[i+244][j]
before2=abs(old_x2-old_y2)
after2=abs(fit_fx2-old_y2)
before_error2=np.nanmean(before2)
after_error2=np.nanmean(after2)
print("September (testing data set) before_error (MADE) : %f" %before_error2)
print("September (testing data set) after_error (MADE) : %f" %after_error2)

September (testing data set) before_error (MADE) : 7.586688
September (testing data set) after_error (MADE) : 5.955634
